In [1]:
import geopandas as gpd
import ee
import geojson
import salem
import pyproj
import shapely
from pyproj import Geod, CRS
import pandas as pd
from geomet import wkt
from shapely import wkt
import sys
from dateutil import parser
import shapefile
import matplotlib.pyplot as plt
import cartopy

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math
import datetime
import seaborn as sns
import os
import platform
import sys
import subprocess
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
#mport branca

#3D stuff
from IPython.core.display import display, HTML
from mpl_toolkits import mplot3d
import json

import geopandas as gpd
import geojson

from functools import partial
#import shapefile
from shapely.geometry import Point # Point class
from shapely.geometry import shape # shape() is a function to convert geo objects through the interface

import cartopy
import cartopy.crs as ccrs
from cartopy.io.img_tiles import OSM
import cartopy.feature as cfeature
from cartopy.io import shapereader
from cartopy.io.img_tiles import StamenTerrain
from cartopy.io.img_tiles import GoogleTiles
from owslib.wmts import WebMapTileService

from matplotlib.path import Path
import matplotlib.patheffects as PathEffects
from matplotlib import patheffects
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

ModuleNotFoundError: No module named 'owslib'

In [4]:
centralasia = gpd.read_file(
    "../data/cropdata/CentralAsia/CAWa_CropType_samples.shp")

durnast = gpd.read_file(
    "../data/cropdata/Bavaria/Test_area.shp")


kenya1 = gpd.read_file(
    '../data/cropdata/Kenya/ref_african_crops_kenya_01_labels/ref_african_crops_kenya_01_labels_00/labels.geojson')
kenya2 = gpd.read_file(
    '../data/cropdata/Kenya/ref_african_crops_kenya_01_labels/ref_african_crops_kenya_01_labels_01/labels.geojson')
kenya3 = gpd.read_file(
    '../data/cropdata/Kenya/ref_african_crops_kenya_01_labels/ref_african_crops_kenya_01_labels_02/labels.geojson')
kenya_merged = pd.concat([kenya1, kenya2, kenya3], axis=0)

bavaria = gpd.read_file(
    "/Volumes/Untitled 1/Sabine Erbe/BY_Antragsshape_2017/Nutzungsschlag_Oberbayern_2017.shp")


# load shapefiles
country = salem.read_shapefile(
    '../data/shapefiles/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp')
counties_france = salem.read_shapefile(
    '../data/shapefiles/FRA_adm/FRA_adm1.shp')
bretagne = counties_france[counties_france.NAME_1 == 'Bretagne']
uzbekistan = country[country.SOVEREIGNT == 'Uzbekistan']
kenya = country[country.SOVEREIGNT == 'Kenya']
bavaria = salem.read_shapefile('../data/shapefiles/bavaria/regbez_ex.shp')



DriverError: ../data/cropdata/Kenya/ref_african_crops_kenya_01_labels/ref_african_crops_kenya_01_labels_00/labels.geojson: No such file or directory

In [ ]:
# get the map from a predefined grid
grid = salem.mercator_grid(transverse=False, center_ll=(15., 28.),
                           extent=(12e6, 8e6))
smap = salem.Map(grid)

# Add the background (other resolutions include: 'mr', 'hr')
# smap.set_rgb(natural_earth='lr')
#smap.set_points(11, 48)
smap.set_shapefile(bavaria, facecolor='deepskyblue', linewidth=0)
smap.set_shapefile(bretagne, facecolor='blue', linewidth=0)
smap.set_shapefile(uzbekistan, facecolor='turquoise', linewidth=0)
smap.set_shapefile(kenya, facecolor='green', linewidth=0)
# done!
smap.set_scale_bar(location=(0.87, 0.04), add_bbox=True)
smap.visualize()
plt.savefig('crop_world.pdf')
plt.show()

In [ ]:
# https://www.net-analysis.com/blog/cartopylayout.html

kenya_merged['Year'] = 2019
kenya_merged.to_file('Kenya_labels/Kenya_labels_PlantVillage.shp')